In [2]:
import os

import pandas as pd
import scipy
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../../Share")
import config, utils, baseline, Processing_same_with_MATLAB


fs = round(10e6 / 2048)  # 4883 Hz
lower_cutoff = 100
upper_cutoff = 600
filter_b, filter_a = Processing_same_with_MATLAB.cheby2(4, 30, [lower_cutoff / (fs/2), upper_cutoff / (fs/2)], btype='bandpass')

feat_mean_1ch = np.array([0.1, 0.1, 2.5, 0.0, 11.0, 229.0, 13.8, -11.0, 9.0, 3.0, 1.5, 0.0, 0.0, 2.8])
feat_std_1ch = np.array([0.02, 0.05, 0.65, 0.02, 4.43, 303.9, 6.85, 12.18, 2.87, 0.87, 0.21, 0.04, 6.68, 1.12])
feat_mean = np.tile(feat_mean_1ch, (4, 1))
feat_std = np.tile(feat_std_1ch, (4, 1))

extractor = Processing_same_with_MATLAB.EMGFeatureExtractor(feat_mean, feat_std, filter_b, filter_a, Norm_bool=False)
trainer = baseline.TremorModelTrainer(config, subject="Hunmin")

# Found that there are normalization applied, so I've tested it without using an normalization

In [3]:
data_files = config.dataset_sub_H
default_path = config.default_path_sub_H

baseline_K = [1, 6, 10, 14, 18, 22, 26, 30, 34, 38]
for K in baseline_K:
    X_train_all, y_train_all, X_test_all, y_test_all = [], [], [], []
    ACC_all = []

    for idx, session_info in enumerate(data_files):
    #for idx, session_info in enumerate(['Exp_2025-06-27-v1/E9AD0E7DCC2B/']):
        #print(f"Dataset {idx + 1}/{len(data_files)} - Session {session_info}\n{'='*40}")
        path = os.path.join(default_path, f'{session_info}raw/')
        features, class_labels = [], []
        for c_idx, c in enumerate(config.classes_5):
            raw_data = os.listdir(path+c)
            mat = scipy.io.loadmat(path+c+raw_data[0])
            extractor.buffer = mat['Data_ADC']
            class_labels.append(mat['Data_Cls'].reshape(-1))
            features_per_cls = extractor.extract_features()
            features_per_cls = np.transpose(features_per_cls, (2, 0, 1))  # shape: (1729, 4, 14)
            features.append(features_per_cls)
            #print(features_per_cls.shape, mat['Data_Cls'].reshape(-1).shape)

        #print(session_info)
        X = np.concatenate(features, axis=0)
        y = np.concatenate(class_labels, axis=0)

        ################################################
        feat_mean = np.mean(X, axis=0)        # shape: (C, F)
        feat_std = np.std(X, axis=0) + 1e-6   # shape: (C, F)
        # Real-time normalization 예시
        X = (X - feat_mean) / feat_std
        ################################################

        if X.shape[0] != y.shape[-1]:
            print(f"Incorrect shape between features and Class: {X.shape} and {y.shape}, {session_info}")
            break

        if idx < K:
            X_train, y_train, X_test, y_test = utils.split_data(X, y, ratio=0.9)
            X_train_all.append(X_train)
            y_train_all.append(y_train)
            X_test_all.append(X_test)
            y_test_all.append(y_test)

            # Concatenate all so far
            X_train_stacked = np.concatenate(X_train_all, axis=0)
            y_train_stacked = np.concatenate(y_train_all, axis=0)
            X_test_stacked = np.concatenate(X_test_all, axis=0)
            y_test_stacked = np.concatenate(y_test_all, axis=0)
            acc=0
        elif idx == K:
            print(f"\t Training {K}: ", X_train_stacked.shape, y_train_stacked.shape)
            acc, pre_trained_CNN = trainer.train_multiple_dataset(X_train, y_train, X_test, y_test)
        else:
            #Test with current data
            X = np.expand_dims(X, axis=-1)
            acc = pre_trained_CNN.evaluate(X, y, verbose=0)[1]*100
            print(f"\t Accuracy on unseen dataset {idx+1}: {acc:.4f}%")
        ACC_all.append(acc)
    pd.DataFrame(ACC_all).to_csv(f'../Results/WithoutNorm/Baseline_results_train_with_{K}data_with_Norm_H_v2.csv', index=False)

	 Training 1:  (7762, 4, 14, 1) (7762,)
Start Training (total epochs: 50)...
Finish Training! (Model is saved)
Maximum training accuracy : 86.23%
Maximum validation accuracy : 91.19%
Accuracy of test dataset using model V0: 90.9618%
	 Accuracy on unseen dataset 3: 76.9096%
	 Accuracy on unseen dataset 4: 83.5713%
	 Accuracy on unseen dataset 5: 82.8932%
	 Accuracy on unseen dataset 6: 71.3938%
	 Accuracy on unseen dataset 7: 66.1350%
	 Accuracy on unseen dataset 8: 72.1830%
	 Accuracy on unseen dataset 9: 71.2227%
	 Accuracy on unseen dataset 10: 70.8884%
	 Accuracy on unseen dataset 11: 71.9776%
	 Accuracy on unseen dataset 12: 71.3968%
	 Accuracy on unseen dataset 13: 73.6922%
	 Accuracy on unseen dataset 14: 72.8281%
	 Accuracy on unseen dataset 15: 80.3661%
	 Accuracy on unseen dataset 16: 84.7420%
	 Accuracy on unseen dataset 17: 84.5304%
	 Accuracy on unseen dataset 18: 84.2523%
	 Accuracy on unseen dataset 19: 87.5773%
	 Accuracy on unseen dataset 20: 88.5024%
	 Accuracy on unse

In [5]:
b_path = '/NCI_mindforce'
baseline_K_char = ['1', '6', '10', '14', '18', '22', '26', '30', '34', '38']
baselines, baselines_N, baselines_N_v2 = [], [], []

for idx, K in enumerate(baseline_K_char):
    baselines.append(b_path + f'/github/Results/Baseline_results_train_with_{K}data_H.csv')
    baselines_N.append(b_path + f'/github/Results/WithoutNorm/Baseline_results_train_with_{K}data_noNorm_H.csv')
    baselines_N_v2.append(b_path + f'/github/Results/WithoutNorm/Baseline_results_train_with_{K}data_with_Norm_H_v2.csv')

for idx, b in enumerate(baseline_K_char):
    ACC_with_norm = float(np.mean(pd.read_csv(baselines[idx])['Accuracy'][int(baseline_K[idx]):]))
    ACC_without_norm = float(np.mean(pd.read_csv(baselines_N[idx])[int(baseline_K[idx]):]))
    ACC_with_norm_but_all = float(np.mean(pd.read_csv(baselines_N_v2[idx])[int(baseline_K[idx]):]))
    print(f"Baseline K-{baseline_K_char[idx]} - Norm : {ACC_with_norm:.2f}% / Without Norm: {ACC_without_norm:.2f}% / With Norm (but all): {ACC_with_norm_but_all:.2f}%")


Baseline K-1 - Norm : 80.61% / Without Norm: 80.27% / With Norm (but all): 77.14%
Baseline K-6 - Norm : 83.63% / Without Norm: 78.35% / With Norm (but all): 83.19%
Baseline K-10 - Norm : 88.47% / Without Norm: 83.80% / With Norm (but all): 85.51%
Baseline K-14 - Norm : 88.19% / Without Norm: 83.83% / With Norm (but all): 84.32%
Baseline K-18 - Norm : 89.09% / Without Norm: 83.24% / With Norm (but all): 87.16%
Baseline K-22 - Norm : 88.91% / Without Norm: 83.86% / With Norm (but all): 84.80%
Baseline K-26 - Norm : 88.78% / Without Norm: 83.56% / With Norm (but all): 83.35%
Baseline K-30 - Norm : 88.26% / Without Norm: 81.77% / With Norm (but all): 84.81%
Baseline K-34 - Norm : 87.24% / Without Norm: 81.28% / With Norm (but all): 83.61%
Baseline K-38 - Norm : 89.57% / Without Norm: 81.41% / With Norm (but all): 82.47%


# Normalization is better
- How to set the normalization value remains a question